In [1]:
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch
import torchvision
from torchvision import datasets, transforms
import torch.utils.data as data
import matplotlib.image as pli
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from PIL import Image

In [2]:
my_transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

file_path = './TinyImageNet'

In [3]:
class ImageSet(data.Dataset):
    def __init__(self):
        self.length = 100 * 1000

    def __getitem__(self, index):
        # print(index)
        label = int(index / 1000)
        img_count = index % 1000
        # print(label)
        # print(img_count)
        img = Image.open(f'{file_path}/train/{label}/{label}_{img_count}.jpg')
        # img.show()
        img = my_transform(img)
        # print(img.size())
        # exit(0)
        return img, label

    def __len__(self):
        return self.length

In [4]:
train_loader = data.DataLoader(ImageSet(), batch_size=256, shuffle=True)
test_loader = data.DataLoader(ImageSet(), batch_size=1, shuffle=True)

In [5]:
class ConvNet(nn.Module):
    def __init__(self,):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            # 64
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=4),
            # 60
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, padding=0)
            # 30
        )
        self.layer2 = nn.Sequential(
            # 30
            nn.Conv2d(in_channels=8, out_channels=16,
                      kernel_size=4),
            # 27
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, padding=0)
            # 9
        )
        self.layer3 = nn.Sequential(
            nn.Linear(in_features=1296, out_features=600),
            nn.ReLU(),
            nn.Linear(in_features=600, out_features=100)
        )

    def forward(self, input):
        out = self.layer1(input)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.layer3(out)
        return out

convNet = ConvNet()
print(convNet)

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(3, 8, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=1296, out_features=600, bias=True)
    (1): ReLU()
    (2): Linear(in_features=600, out_features=100, bias=True)
  )
)


In [6]:
# 加载模型， 请谨慎操作， 会覆盖在内存中的模型
convNet = ConvNet()
convNet.load_state_dict(torch.load('./ConvNet.model'))
convNet.eval()

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(3, 8, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=1296, out_features=600, bias=True)
    (1): ReLU()
    (2): Linear(in_features=600, out_features=100, bias=True)
  )
)

In [7]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convNet.parameters(), lr=0.01)


In [8]:
for i, (images, labels) in enumerate(train_loader):
    outputs = convNet.forward(images)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    predict = torch.argmax(nn.functional.softmax(outputs, dim=1), dim=1)
    print(f"i = {i},  accuracy = {sum(labels == predict)/labels.size(0)}")

i = 0,  accuracy = 0.01171875
i = 1,  accuracy = 0.00390625
i = 2,  accuracy = 0.01171875
i = 3,  accuracy = 0.015625
i = 4,  accuracy = 0.0
i = 5,  accuracy = 0.015625
i = 6,  accuracy = 0.015625
i = 7,  accuracy = 0.0
i = 8,  accuracy = 0.01953125
i = 9,  accuracy = 0.0078125
i = 10,  accuracy = 0.0078125
i = 11,  accuracy = 0.015625
i = 12,  accuracy = 0.015625
i = 13,  accuracy = 0.01953125
i = 14,  accuracy = 0.01171875
i = 15,  accuracy = 0.01953125
i = 16,  accuracy = 0.0234375
i = 17,  accuracy = 0.0078125
i = 18,  accuracy = 0.01171875
i = 19,  accuracy = 0.0078125
i = 20,  accuracy = 0.01171875
i = 21,  accuracy = 0.01171875
i = 22,  accuracy = 0.0234375
i = 23,  accuracy = 0.0078125
i = 24,  accuracy = 0.0390625
i = 25,  accuracy = 0.02734375
i = 26,  accuracy = 0.0234375
i = 27,  accuracy = 0.01953125
i = 28,  accuracy = 0.01171875
i = 29,  accuracy = 0.0234375
i = 30,  accuracy = 0.01953125
i = 31,  accuracy = 0.01953125
i = 32,  accuracy = 0.015625
i = 33,  accuracy = 0.0

KeyboardInterrupt: 

In [10]:
# 保存模型， 请谨慎操作， 会覆盖文件中的模型
torch.save(convNet.state_dict(), './ConvNet.model')